### Project : Sentiments classification on movies review dataset

In [2]:
import numpy as np
import re
import pickle
#import nltk
#from nltk.corpus import stopwords
from sklearn.datasets import load_files
import pandas as pd
#load_files:Load text files with categories as subfolder names.

### clean data

In [3]:
def clean(x):
    #x=re.sub(r'\W',' ',x)
    #x = re.sub(r'[^a-zA-Z]',' ',x)
    x = re.sub("wouldn\'t",'would not',x)
    x = re.sub("they \ 've",'they have',x)
    
    #to remove html tags
    x = re.sub(r'<.*?>', '', x)
    
    #to remove everything except alpha
    x = re.sub(r'[^a-zA-Z]',' ',x)
    
      
    x = re.sub(r'\s+',' ',x)          #remove extra space's
    return x.lower()
    
#\W:matches any non-alphanumeric character; 
#this is equivalent to the set [^a-zA-Z0-9_].

### Convert text into numeric

In [4]:

from nltk.corpus import stopwords
import nltk

In [5]:

words = stopwords.words('english')


In [6]:
if 'not' in words:
    words.remove('not')

In [7]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

In [8]:
#cv = CountVectorizer(min_df=10,max_df=.6,stop_words=words)
cv = CountVectorizer(stop_words=words)

#min_df=10:exclude any word that comes in 10 or less than 10 documents
#max_df=.6:excude any word that comes more than 60% of the documents,

In [9]:

from sklearn.naive_bayes import MultinomialNB

In [10]:
nb = MultinomialNB()


### Lets work on 50000 samples

In [11]:
df = pd.read_csv('movie_reviews.csv')

In [12]:
df.head()

,review,sentiment
0,This is one of those unfortunate films that su...,1
1,Okay maybe it was because I happen to be in Ya...,1
2,"Although I love this movie, I can barely watch...",1
3,"A man arrives in a strange, beautiful, sterile...",1
4,I'm sitting around going through movie listing...,1


In [13]:
df.shape

(50000, 2)

In [14]:
df.sentiment.value_counts()

0    25000
1    25000
Name: sentiment, dtype: int64

In [15]:
df['review'] = df.review.apply(clean)

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
x_train,x_test,y_train,y_test = train_test_split(df.review.values,
                                                 df.sentiment.values,
                                                test_size=10000,
                                                random_state=10)

In [18]:
x_train.shape

(40000,)

In [19]:
np.bincount(y_test)

array([4962, 5038], dtype=int64)

In [20]:
cv1 = CountVectorizer(stop_words=words)


In [21]:
#x_new=cv1.fit_transform(x_train).toarray()
#x_test_new = cv1.transform(x_test).toarray()

x_new=cv1.fit_transform(x_train)
x_test_new = cv1.transform(x_test)

In [22]:
x_new[0].toarray()

array([[0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [23]:
x_new.shape

(40000, 92358)

In [24]:
x_test_new.shape

(10000, 92358)

In [25]:
nb = MultinomialNB()
nb.fit(x_new,y_train)

MultinomialNB()

In [26]:
nb.score(x_test_new,y_test)

0.862

In [27]:
test=["I do not like this movie",
      "This movie is adorable",
     "I hate this movie",
      "I love this movie"]

In [28]:
f=[]
for i in test:
    s=clean(i)
    f.append(s)

In [29]:
f

['i do not like this movie',
 'this movie is adorable',
 'i hate this movie',
 'i love this movie']

In [30]:
t=cv1.transform(f).toarray()

In [31]:
t.shape

(4, 92358)

In [32]:
nb.predict(t)

array([0, 1, 0, 1], dtype=int64)

### save the model

In [33]:
with open('nb_model.pkl','wb') as f1:
    pickle.dump(nb,f1)

In [34]:
### save the vectorizer
with open('cv1.pkl','wb') as f1:
    pickle.dump(cv1,f1)

In [35]:
### load model

In [36]:
with open('nb_model.pkl','rb') as f1:
    clf=pickle.load(f1)

In [37]:
clf.predict(t)

array([0, 1, 0, 1], dtype=int64)